In [40]:
import pandas as pd
import re
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
# Cargar los archivos CSV y Eliminacion de Columnas Vacias.

df_MasterProductos = pd.read_csv("L_STMaestroPrd.csv")  # Tu archivo maestro

# Eliminar las columnas que están completamente vacías
df_MasterProductos = df_MasterProductos.dropna(axis=1, how='all')

# Crear la nueva columna 'descrip1_descrip2' combinando las columnas 'descrip1' y 'descrip2'
df_MasterProductos["descrip1_descrip2_MasterProductos"] = df_MasterProductos["descrip1"].fillna('') + " " + df_MasterProductos["descrip2"].fillna('')

# Mostrar el resultado
df_MasterProductos


C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_14768\372272267.py:3: DtypeWarning: Columns (7,14,17,18,21,28,39,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df_MasterProductos = pd.read_csv("L_STMaestroPrd.csv")  # Tu archivo maestro


,niprod,cod_prod,cod_abrev,descrip1,descrip2,abrev,cod_catprod,codbarra,cod_fbarra,unid_v,...,tipo_egreso,pc_diftrans,cod_cvta,cat_item,cant_rep,pto_rep,atrib0,atrib1,atrib2,descrip1_descrip2_MasterProductos
0,122703.0,MEE-EST-00904,12185,-,-,-,EM,NaN,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,JAYOR,NaN,NaN,- -
1,134925.0,APA-BAL-03444,-,-,-,-,MP,NaN,NaN,UN,...,F,0.0,NaN,ITG,0,0.0,NaN,NaN,NaN,- -
2,124372.0,APE-00080,SUPP500FRE,(REPETIDO) SUPPORTAN 500ML,CAJA X 15 SACHET X 500ML - UM: SACHET,SUPP500FRE,AL,NaN,NaN,UN,...,E,0.0,NaN,ITG,0,0.0,NaN,NaN,NaN,(REPETIDO) SUPPORTAN 500ML CAJA X 15 SACHET X ...
3,125531.0,DEC-PGM-03064,48457,BOLSAS PAPEL GRADO MEDICO 175X65X420MM - KIMS,BOLSA X 500 UNIDADES UM: BOLSA,PAPEL GRADOMEDICO 17,MP,NaN,NaN,UN,...,E,0.0,NaN,ITG,0,0.0,ETILEN,NaN,NaN,BOLSAS PAPEL GRADO MEDICO 175X65X420MM - KIMS ...
4,133202.0,DEC-PGM-03110,EPBL-19,BOLSAS PAPEL GRADO MEDICO 260X60X520MM - KIMS,PACK X 1000 UNIDADES UM: PACK,16/08/24,MP,NaN,NaN,UN,...,E,0.0,NaN,ITG,0,0.0,ETILEN,NaN,NaN,BOLSAS PAPEL GRADO MEDICO 260X60X520MM - KIMS ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56769,118010.0,49702,49702,ZYVALIX 250 mg comp.x 120,NaN,ZYVALIX 250 mg comp.,PT,7795367001885.0,LKM_B,UN,...,E,0.0,NaN,ITGEV,0,0.0,LKM ONCO/ESPECI,NaN,NaN,ZYVALIX 250 mg comp.x 120
56770,129759.0,61433,61433,ZYVALIX PLUS 500 mg comp.x 60,NaN,ZYVALIX PLUS 500 mg,PT,7795367552196.0,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,LKM ONCO/ESPECI,NaN,NaN,ZYVALIX PLUS 500 mg comp.x 60
56771,26456.0,27450,27450,ZYVOX IV 600 mg bolsas x 10x300ml,CAJA X 10 BOLSAS X 300ML - UM: CAJA,ZYVOX IV 600 mg bols,PT,7791824117007.0,BOEHRINGER,UN,...,E,0.0,NaN,ITGEV,0,0.0,PFIZER,NaN,NaN,ZYVOX IV 600 mg bolsas x 10x300ml CAJA X 10 BO...
56772,26457.0,27451,27451,ZYVOX ORAL 600 mg tab.x 10,NaN,ZYVOX ORAL 600 mg ta,PT,7791824116994.0,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PFIZER,NaN,NaN,ZYVOX ORAL 600 mg tab.x 10


In [42]:
import re

#PROCESAMIENTO DE DATOS EN LA COLUMNA DESCRIPCION 

def procesar_descripcion(data, nombre_columna):
    # Función para normalizar la descripción
    def normalizar(texto):
        # 1. Eliminar espacios innecesarios y homogeneizar texto
        texto = texto.strip().replace(r'\s+', ' ').lower()

        # 2. Remover caracteres especiales
        texto = re.sub(r'[^a-zA-Z0-9\s%]', '', texto)

        # 3. Separar números de letras (ej: 420mm -> 420 mm)
        texto = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', texto)
        texto = re.sub(r'([a-zA-Z])(\d)', r'\1 \2', texto)

        # 4. Agregar espacios alrededor de la 'x' usada como separador (ej: 10x10 -> 10 x 10)
        texto = re.sub(r'(?<=\d)x(?=\d)', r' x ', texto)

        # 5. Generalizar la normalización de términos combinados que terminan en 'x' (ej: tabx -> tab x)
        texto = re.sub(r'(\b\w+)x\b', r'\1 x', texto)

        # 6. Reemplazar separadores como guiones o barras con espacios
        texto = re.sub(r'[-/]', ' ', texto)

        # 7. Remover espacios redundantes
        texto = re.sub(r'\s+', ' ', texto).strip()

        # 8. Normalizar términos específicos combinados (ej: tabx -> tab x)
        texto = re.sub(r'\b(tabx)\b', r'tab x', texto)
        texto = re.sub(r'\b(compx)\b', r'comp x', texto)
        texto = re.sub(r'\b(locx)\b', r'loc x', texto)
        texto = re.sub(r'\b(comprecx)\b', r'comprec x', texto)
        texto = re.sub(r'\b(capsx)\b', r'caps x', texto)
        texto = re.sub(r'\b(jabx)\b', r'jab x', texto)
        texto = re.sub(r'\b(pdax)\b', r'pda x', texto)
        texto = re.sub(r'\b(pvox)\b', r'pvo x', texto)
        texto = re.sub(r'\b(fcocvlvdosifx)\b', r'fcocvlvdosif x', texto)
        texto = re.sub(r'\b(sobx)\b', r'sob x', texto)
        texto = re.sub(r'\b(crx)\b', r'cr x', texto)

        # 9. Manejo de Números (remover espacios alrededor de 'x' en expresiones como 10 x 10)
        texto = re.sub(r'(\d+)\s*[xX]\s*(\d+)', r'\1x\2', texto)

        # 10. Generalizar manejo de unidades (ej: "10mg" y "10 mg" -> "10 mg", "250ml" y "250 ml" -> "250 ml")
        texto = re.sub(r'(\d+)\s*([a-zA-Z]+)', r'\1 \2', texto)

        # 11. Eliminar palabras repetidas en la misma fila (como "paracetamol paracetamol")
        texto = ' '.join(sorted(set(texto.split()), key=texto.split().index))

        return texto

    # Aplicar la función 'normalizar' a cada elemento de la columna
    data[nombre_columna] = data[nombre_columna].apply(normalizar)
    
    return data


In [43]:
#PROCESAMIENTO DE DATOS DE LA COLUMNA descrip1 y descrip2 1 DE LA MASTER 
# Asegúrate de que el DataFrame de salida esté correctamente definido
data_MasterProductos_Procesada = df_MasterProductos.copy()

nombre_columna = "descrip1_descrip2_MasterProductos"
# Aplicar la función de procesamiento y guardar el resultado en una nueva columna con el nombre deseado

data_MasterProductos_Procesada["descrip1_descrip2_MasterProductos"] = procesar_descripcion(df_MasterProductos, nombre_columna)["descrip1_descrip2_MasterProductos"]

# Ahora puedes acceder a la columna original procesada o la nueva columna
data_MasterProductos_Procesada


,niprod,cod_prod,cod_abrev,descrip1,descrip2,abrev,cod_catprod,codbarra,cod_fbarra,unid_v,...,tipo_egreso,pc_diftrans,cod_cvta,cat_item,cant_rep,pto_rep,atrib0,atrib1,atrib2,descrip1_descrip2_MasterProductos
0,122703.0,MEE-EST-00904,12185,-,-,-,EM,NaN,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,JAYOR,NaN,NaN,
1,134925.0,APA-BAL-03444,-,-,-,-,MP,NaN,NaN,UN,...,F,0.0,NaN,ITG,0,0.0,NaN,NaN,NaN,
2,124372.0,APE-00080,SUPP500FRE,(REPETIDO) SUPPORTAN 500ML,CAJA X 15 SACHET X 500ML - UM: SACHET,SUPP500FRE,AL,NaN,NaN,UN,...,E,0.0,NaN,ITG,0,0.0,NaN,NaN,NaN,repetido supportan 500 ml caja x 15 sachet um
3,125531.0,DEC-PGM-03064,48457,BOLSAS PAPEL GRADO MEDICO 175X65X420MM - KIMS,BOLSA X 500 UNIDADES UM: BOLSA,PAPEL GRADOMEDICO 17,MP,NaN,NaN,UN,...,E,0.0,NaN,ITG,0,0.0,ETILEN,NaN,NaN,bolsas papel grado medico 175 x65 x 420 mm kim...
4,133202.0,DEC-PGM-03110,EPBL-19,BOLSAS PAPEL GRADO MEDICO 260X60X520MM - KIMS,PACK X 1000 UNIDADES UM: PACK,16/08/24,MP,NaN,NaN,UN,...,E,0.0,NaN,ITG,0,0.0,ETILEN,NaN,NaN,bolsas papel grado medico 260 x60 x 520 mm kim...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56769,118010.0,49702,49702,ZYVALIX 250 mg comp.x 120,NaN,ZYVALIX 250 mg comp.,PT,7795367001885.0,LKM_B,UN,...,E,0.0,NaN,ITGEV,0,0.0,LKM ONCO/ESPECI,NaN,NaN,zyvali x 250 mg comp 120
56770,129759.0,61433,61433,ZYVALIX PLUS 500 mg comp.x 60,NaN,ZYVALIX PLUS 500 mg,PT,7795367552196.0,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,LKM ONCO/ESPECI,NaN,NaN,zyvali x plus 500 mg comp 60
56771,26456.0,27450,27450,ZYVOX IV 600 mg bolsas x 10x300ml,CAJA X 10 BOLSAS X 300ML - UM: CAJA,ZYVOX IV 600 mg bols,PT,7791824117007.0,BOEHRINGER,UN,...,E,0.0,NaN,ITGEV,0,0.0,PFIZER,NaN,NaN,zyvo x iv 600 mg bolsas 10 x300 ml caja 300 um
56772,26457.0,27451,27451,ZYVOX ORAL 600 mg tab.x 10,NaN,ZYVOX ORAL 600 mg ta,PT,7791824116994.0,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PFIZER,NaN,NaN,zyvo x oral 600 mg tab 10


In [44]:
# Este modulo selecciona que algoritmo usar para estandarizar los csv que provienen del proveedor.

# Función para procesar el caso de "varios productos en una fila"
def procesar_varios_productos(df):
    for column in df.columns:
        df[column] = df[column].apply(lambda x: x.split('\n') if isinstance(x, str) else ([x] if pd.notna(x) else [np.nan]))
    df_exploded_list = []
    for _, row in df.iterrows():
        row_lists = [row[col] if isinstance(row[col], list) else [row[col]] for col in df.columns]
        max_len = max(len(lst) for lst in row_lists)
        row_lists = [lst + [None] * (max_len - len(lst)) for lst in row_lists]
        exploded_row = pd.DataFrame(row_lists).T
        exploded_row.columns = df.columns
        df_exploded_list.append(exploded_row)
    return pd.concat(df_exploded_list, ignore_index=True)

# Función para procesar el caso de "un producto por fila"
def procesar_un_producto(df):
    for col in df.columns:
        df[col] = df[col].apply(lambda x: x.replace('\r\n', ' ').replace('\n', ' ') if isinstance(x, str) else x)
    return df

# Función principal para decidir el módulo a usar
def procesar_csv(file_path):
    df = pd.read_csv(file_path)
    contiene_saltos = df.iloc[:, 0].apply(lambda x: '\n' in x if isinstance(x, str) else False).any()
    if contiene_saltos:
        print("Usando el módulo: VARIOS PRODUCTOS EN UNA FILA")
        return procesar_varios_productos(df)
    else:
        print("Usando el módulo: UN PRODUCTO POR FILA")
        return procesar_un_producto(df)

# Cargar el CSV del proveedor procesado 
ruta_csv = "prueba2.csv"
df_resultado_PROVEEDOR = procesar_csv(ruta_csv)
df_resultado_PROVEEDOR


# Copiamos el DataFrame original
df_resultado_PROVEEDOR = df_resultado_PROVEEDOR.copy()

# Asegurarse de manejar valores nulos antes de concatenar
df_resultado_PROVEEDOR["descrip1_descrip2_PROVEEDOR"] = df_resultado_PROVEEDOR["DESCRIPCIÓN"].fillna('') + " " + df_resultado_PROVEEDOR["PRESENTACION"].fillna('')
df_resultado_PROVEEDOR["descrip1_descrip2_PROVEEDOR"] = df_resultado_PROVEEDOR["descrip1_descrip2_PROVEEDOR"].str.strip()  # Eliminar espacios en los extremos

df_resultado_PROVEEDOR


Usando el módulo: VARIOS PRODUCTOS EN UNA FILA


,Unnamed: 0,DESCRIPCIÓN,PRESENTACION,CODIGO DE BARRAS,PRECIO CON IVA,PRECIO SIN IVA,PUBLICO\nSUGERIDO,descrip1_descrip2_PROVEEDOR
0,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",8054083017068,"$ 740,145.59","$ 611,690.58","$ 1,073,211.02","HUMIRA AC x 2 (20mg) 2 JERINGAS PRELL. X 0,2 ml"
1,20005289,HUMIRA AC x 2 (40mg),"2 LAPICERAS PRELL. X 0,4 ml",8054083014050,"$ 1,480,292.66","$ 1,223,382.36","$ 2,146,424.35","HUMIRA AC x 2 (40mg) 2 LAPICERAS PRELL. X 0,4 ml"
2,20021980,HUMIRA AC x 1 (80 mg),"1 LAPICERA PRELL. X 0,8 ml",8054083017648,"$ 1,553,789.84","$ 1,284,123.82","$ 2,252,995.41","HUMIRA AC x 1 (80 mg) 1 LAPICERA PRELL. X 0,8 ml"
3,20062091,RINVOQ 15MG,15 MG x 30 COMP REC DE LIB. PROLONGADA,8054083018706,"$ 1,799,606.71","$ 1,487,278.28","$ 2,609,429.58",RINVOQ 15MG 15 MG x 30 COMP REC DE LIB. PROLO...
4,20069005,RINVOQ 30MG,30 MG x 30 COMP REC DE LIB. PROLONGADA,8054083021386,"$ 3,184,521.02","$ 2,631,835.62","$ 4,617,555.55",RINVOQ 30MG 30 MG x 30 COMP REC DE LIB. PROLON...
5,20078725,RINVOQ 45MG,45 MG x 28 COMP REC DE LIB. PROLONGADA,8054083023922,"$ 4,494,517.30","$ 3,714,477.10","$ 6,517,049.97",RINVOQ 45MG 45 MG x 28 COMP REC DE LIB. PROLON...
6,20069017,SKYRIZI,1 LAPICERA PRELLENADA por 1 ML (150 mg),8054083021935,"$ 4,344,159.74","$ 3,590,214.67","$ 6,299,031.64",SKYRIZI 1 LAPICERA PRELLENADA por 1 ML (150 mg)
7,20079636,SKYRIZI,360mg /2.4ml Cart. prellenado x 1 600mg /10ml ...,8054083023311,"$ 4,561,367.72","$ 3,769,725.40","$ 6,613,983.22",SKYRIZI 360mg /2.4ml Cart. prellenado x 1 600m...
8,20079635,SKYRIZI,None,8054083023328,"$ 4,561,367.72","$ 3,769,725.40","$ 6,613,983.22",SKYRIZI
9,20058578,KALETRA SOLUCIÓN,160 ML,8054083006406,"$ 148,177.93","$ 122,461.09","$ 214,857.98",KALETRA SOLUCIÓN 160 ML


In [45]:

data_Proveedor_Procesada = df_resultado_PROVEEDOR.copy()

nombre_columna = "descrip1_descrip2_PROVEEDOR"
# Aplicar la función de procesamiento y guardar el resultado en una nueva columna con el nombre deseado

data_Proveedor_Procesada["descrip1_descrip2_PROVEEDOR"] = procesar_descripcion(data_Proveedor_Procesada, nombre_columna)["descrip1_descrip2_PROVEEDOR"]

# Ahora puedes acceder a la columna original procesada o la nueva columna
data_Proveedor_Procesada


,Unnamed: 0,DESCRIPCIÓN,PRESENTACION,CODIGO DE BARRAS,PRECIO CON IVA,PRECIO SIN IVA,PUBLICO\nSUGERIDO,descrip1_descrip2_PROVEEDOR
0,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",8054083017068,"$ 740,145.59","$ 611,690.58","$ 1,073,211.02",humira ac x 2 20 mg jeringas prell 02 ml
1,20005289,HUMIRA AC x 2 (40mg),"2 LAPICERAS PRELL. X 0,4 ml",8054083014050,"$ 1,480,292.66","$ 1,223,382.36","$ 2,146,424.35",humira ac x 2 40 mg lapiceras prell 04 ml
2,20021980,HUMIRA AC x 1 (80 mg),"1 LAPICERA PRELL. X 0,8 ml",8054083017648,"$ 1,553,789.84","$ 1,284,123.82","$ 2,252,995.41",humira ac x 1 80 mg lapicera prell 08 ml
3,20062091,RINVOQ 15MG,15 MG x 30 COMP REC DE LIB. PROLONGADA,8054083018706,"$ 1,799,606.71","$ 1,487,278.28","$ 2,609,429.58",rinvoq 15 mg x 30 comp rec de lib prolongada
4,20069005,RINVOQ 30MG,30 MG x 30 COMP REC DE LIB. PROLONGADA,8054083021386,"$ 3,184,521.02","$ 2,631,835.62","$ 4,617,555.55",rinvoq 30 mg x comp rec de lib prolongada
5,20078725,RINVOQ 45MG,45 MG x 28 COMP REC DE LIB. PROLONGADA,8054083023922,"$ 4,494,517.30","$ 3,714,477.10","$ 6,517,049.97",rinvoq 45 mg x 28 comp rec de lib prolongada
6,20069017,SKYRIZI,1 LAPICERA PRELLENADA por 1 ML (150 mg),8054083021935,"$ 4,344,159.74","$ 3,590,214.67","$ 6,299,031.64",skyrizi 1 lapicera prellenada por ml 150 mg
7,20079636,SKYRIZI,360mg /2.4ml Cart. prellenado x 1 600mg /10ml ...,8054083023311,"$ 4,561,367.72","$ 3,769,725.40","$ 6,613,983.22",skyrizi 360 mg 24 ml cart prellenado x 1 600 1...
8,20079635,SKYRIZI,None,8054083023328,"$ 4,561,367.72","$ 3,769,725.40","$ 6,613,983.22",skyrizi
9,20058578,KALETRA SOLUCIÓN,160 ML,8054083006406,"$ 148,177.93","$ 122,461.09","$ 214,857.98",kaletra solucin 160 ml


In [46]:
# Función para filtrar productos por laboratorios sobre la MASTER PRODUCTOS 
def laboratorio_input_df(df, columna):
    input_name = input("Ingrese el nombre del laboratorio: ").strip()
    tablas_coincidencias = df[df[columna].str.contains(input_name, case=False, na=False)].copy()
    if not tablas_coincidencias.empty:
        print(f"Se encontraron {len(tablas_coincidencias)} productos coincidentes del laboratorio '{input_name}'.")
        return tablas_coincidencias
    else:
        print(f"No se encontraron productos del laboratorio '{input_name}'.")
        return None

# Usar la función para filtrar productos
column_name_laboratorio = "atrib0"  # Columna donde se busca el laboratorio
df_Laboratorio_MasterProducto_Filtrado = laboratorio_input_df(data_MasterProductos_Procesada, column_name_laboratorio)

data_Laboratorio_MasterProducto_Procesado_Filtrado = df_Laboratorio_MasterProducto_Filtrado[[col for col in df_Laboratorio_MasterProducto_Filtrado.columns if col != 'descrip1_descrip2_MasterProductos'][:3] + ['descrip1_descrip2_MasterProductos'] + [col for col in df_Laboratorio_MasterProducto_Filtrado.columns if col != 'descrip1_descrip2_MasterProductos'][3:]]

# Ver el resultado
data_Laboratorio_MasterProducto_Procesado_Filtrado


Se encontraron 100 productos coincidentes del laboratorio 'abbv'.


,niprod,cod_prod,cod_abrev,descrip1_descrip2_MasterProductos,descrip1,descrip2,abrev,cod_catprod,codbarra,cod_fbarra,...,ninum,tipo_egreso,pc_diftrans,cod_cvta,cat_item,cant_rep,pto_rep,atrib0,atrib1,atrib2
787,23422.0,38945,38945,acular ls soloft x 5 ml,ACULAR LS sol.oft.x 5 ml,NaN,ACULAR LS sol.oft.x,PT,8054083026008,NaN,...,NaN,E,0.0,NaN,ITGEV,0,0.0,ABBVIE ALLERGAN,NaN,NaN
788,23421.0,15467,15467,acular soloft x 5 ml,ACULAR sol.oft.x 5 ml,NaN,ACULAR sol.oft.x 5 m,PT,8054083026329,NaN,...,NaN,E,0.0,NaN,ITGEV,0,0.0,ABBVIE ALLERGAN,NaN,NaN
790,23423.0,48009,48009,acuvail viales unids x 30 x04 ml,ACUVAIL viales unids.x 30 x0.4ml,NaN,ACUVAIL viales unids,PT,7795316000099,NaN,...,NaN,E,0.0,NaN,ITGEV,0,0.0,ABBVIE ALLERGAN,NaN,NaN
1654,23424.0,44172,44172,aibre x soloft 5 ml,AIBREX sol.oft.x 5 ml,NaN,AIBREX sol.oft.x 5 m,PT,7795316000112,NaN,...,NaN,E,0.0,NaN,ITGEV,0,0.0,ABBVIE ALLERGAN,NaN,NaN
2439,23427.0,31189,31189,alphagan p soloft x 5 ml,ALPHAGAN P sol.oft.x 5 ml,NaN,ALPHAGAN P sol.oft.x,PT,8054083026312,NaN,...,NaN,E,0.0,NaN,ITGEV,0,0.0,ABBVIE ALLERGAN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56458,119578.0,39616,39616,zemplar 5 mcg a x x1 ml,ZEMPLAR 5 mcg a.x 5 x 1 ml,NaN,ZEMPLAR 5 mcg a.x 5,PT,8054083003566.0,ABBOTT6,...,101.0,E,0.0,NaN,ITGEV,0,0.0,ABBVIE,NaN,NaN
56459,119579.0,54285,54285,zemplar 5 mcg vial x x1 ml,ZEMPLAR 5 mcg vial x 5 x 1 ml,NaN,ZEMPLAR 5 mcg vial x,PT,8054083017907.0,NaN,...,NaN,E,0.0,NaN,ITGEV,0,0.0,ABBVIE,NaN,NaN
56745,23480.0,34168,34168,zymaran 03% soloft x 5 ml,ZYMARAN 0.3% sol.oft.x 5 ml,NaN,ZYMARAN 0.3% sol.oft,PT,8054083026336.0,NaN,...,NaN,E,0.0,NaN,ITGEV,0,0.0,ABBVIE ALLERGAN,NaN,NaN
56746,23481.0,47867,47867,zymaxid fcogotero x 5 ml,ZYMAXID fco.gotero x 5 ml,NaN,ZYMAXID fco.gotero x,PT,8054083026350.0,NaN,...,NaN,E,0.0,NaN,ITGEV,0,0.0,ABBVIE ALLERGAN,NaN,NaN


In [47]:
#DATA FRAME DE SENTENCE TRANSFORMER 
# Master Prdocuto Filtrada por Laboratorio.
data_Laboratorio_MasterProducto_Procesado_Filtrado 

#Proveedor CSV procesado
data_Proveedor_Procesada


,Unnamed: 0,DESCRIPCIÓN,PRESENTACION,CODIGO DE BARRAS,PRECIO CON IVA,PRECIO SIN IVA,PUBLICO\nSUGERIDO,descrip1_descrip2_PROVEEDOR
0,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",8054083017068,"$ 740,145.59","$ 611,690.58","$ 1,073,211.02",humira ac x 2 20 mg jeringas prell 02 ml
1,20005289,HUMIRA AC x 2 (40mg),"2 LAPICERAS PRELL. X 0,4 ml",8054083014050,"$ 1,480,292.66","$ 1,223,382.36","$ 2,146,424.35",humira ac x 2 40 mg lapiceras prell 04 ml
2,20021980,HUMIRA AC x 1 (80 mg),"1 LAPICERA PRELL. X 0,8 ml",8054083017648,"$ 1,553,789.84","$ 1,284,123.82","$ 2,252,995.41",humira ac x 1 80 mg lapicera prell 08 ml
3,20062091,RINVOQ 15MG,15 MG x 30 COMP REC DE LIB. PROLONGADA,8054083018706,"$ 1,799,606.71","$ 1,487,278.28","$ 2,609,429.58",rinvoq 15 mg x 30 comp rec de lib prolongada
4,20069005,RINVOQ 30MG,30 MG x 30 COMP REC DE LIB. PROLONGADA,8054083021386,"$ 3,184,521.02","$ 2,631,835.62","$ 4,617,555.55",rinvoq 30 mg x comp rec de lib prolongada
5,20078725,RINVOQ 45MG,45 MG x 28 COMP REC DE LIB. PROLONGADA,8054083023922,"$ 4,494,517.30","$ 3,714,477.10","$ 6,517,049.97",rinvoq 45 mg x 28 comp rec de lib prolongada
6,20069017,SKYRIZI,1 LAPICERA PRELLENADA por 1 ML (150 mg),8054083021935,"$ 4,344,159.74","$ 3,590,214.67","$ 6,299,031.64",skyrizi 1 lapicera prellenada por ml 150 mg
7,20079636,SKYRIZI,360mg /2.4ml Cart. prellenado x 1 600mg /10ml ...,8054083023311,"$ 4,561,367.72","$ 3,769,725.40","$ 6,613,983.22",skyrizi 360 mg 24 ml cart prellenado x 1 600 1...
8,20079635,SKYRIZI,None,8054083023328,"$ 4,561,367.72","$ 3,769,725.40","$ 6,613,983.22",skyrizi
9,20058578,KALETRA SOLUCIÓN,160 ML,8054083006406,"$ 148,177.93","$ 122,461.09","$ 214,857.98",kaletra solucin 160 ml


In [48]:
# Instanciamos el modelo de SentenceTransformer para obtener embeddings de texto
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Función para obtener embeddings de las descripciones
def get_embeddings(descriptions):
    return model.encode(descriptions.tolist())

# Comparar descripciones de productos utilizando embeddings
"""
    DESCRIPCIÓN: proviene del CSV PROVEEDOR 
    descrip1: proviene del CSV FILTRADO DE LA MASTER
"""

if data_Laboratorio_MasterProducto_Procesado_Filtrado is not None:
    proveedor_embeddings = get_embeddings(data_Proveedor_Procesada['descrip1_descrip2_PROVEEDOR'])
    laboratorio_embeddings = get_embeddings(data_Laboratorio_MasterProducto_Procesado_Filtrado['descrip1_descrip2_MasterProductos'])

    matches = []

    for i, prov_row in data_Proveedor_Procesada.iterrows():
        prov_desc = prov_row['descrip1_descrip2_PROVEEDOR']

        # Calculamos la similitud de coseno entre el producto proveedor y los productos del laboratorio
        cosine_similarities = cosine_similarity([proveedor_embeddings[i]], laboratorio_embeddings)

        best_match_index = cosine_similarities.argmax()  # El índice de la mejor coincidencia
        best_match_score = cosine_similarities.max()  # El puntaje de similitud

        match_info = prov_row.to_dict()
        match_info.update(data_Laboratorio_MasterProducto_Procesado_Filtrado.iloc[best_match_index].to_dict())
        match_info['similaridad'] = best_match_score
        matches.append(match_info)

    df_matches = pd.DataFrame(matches)

    # Reordenamos las columnas para que la columna 'similaridad' esté al inicio
    cols = ['similaridad'] + [col for col in df_matches.columns if col != 'similaridad']
    df_matches = df_matches[cols]

    print(f"\nCoincidencias encontradas: {len(df_matches[df_matches['similaridad'] > 0])}")
    print(df_matches)

    productos_sin_coincidencia = data_Laboratorio_MasterProducto_Procesado_Filtrado[~data_Laboratorio_MasterProducto_Procesado_Filtrado['descrip1_descrip2_MasterProductos'].isin(
        df_matches[df_matches['similaridad'] > 0]['descrip1_descrip2_MasterProductos']
    )]

    print(f"\nProductos del laboratorio sin coincidencias: {len(productos_sin_coincidencia)}")
    print(productos_sin_coincidencia)
else:
    print("No se encontraron coincidencias para el laboratorio especificado.")

df_matches


Coincidencias encontradas: 27
    similaridad Unnamed: 0                  DESCRIPCIÓN  \
0      0.911634   20015888         HUMIRA AC x 2 (20mg)   
1      0.932588   20005289        HUMIRA AC x 2 (40mg)    
2      0.951788   20021980        HUMIRA AC x 1 (80 mg)   
3      0.601083   20062091                 RINVOQ 15MG    
4      0.593531   20069005                  RINVOQ 30MG   
5      0.623665   20078725                  RINVOQ 45MG   
6      0.776140   20069017                      SKYRIZI   
7      0.886561   20079636                      SKYRIZI   
8      0.582979   20079635                      SKYRIZI   
9      0.692462   20058578             KALETRA SOLUCIÓN   
10     0.643237   20025520         KALETRA COMPRIMIDOS    
11     0.670539   20012232          KALETRA COMPRIMIDOS   
12     0.810732   20011289                    RITONAVIR   
13     0.753797   20011756                      MAVIRET   
14     0.641584   20019068   2 JERINGAS PRELL. X 0,2 ml   
15     0.570240   2001906

,similaridad,Unnamed: 0,DESCRIPCIÓN,PRESENTACION,CODIGO DE BARRAS,PRECIO CON IVA,PRECIO SIN IVA,PUBLICO\nSUGERIDO,descrip1_descrip2_PROVEEDOR,niprod,...,ninum,tipo_egreso,pc_diftrans,cod_cvta,cat_item,cant_rep,pto_rep,atrib0,atrib1,atrib2
0,0.911634,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",8054083017068,"$ 740,145.59","$ 611,690.58","$ 1,073,211.02",humira ac x 2 20 mg jeringas prell 02 ml,119551.0,...,101.0,E,0.0,NaN,ITGEV,0,0.0,ABBVIE,NaN,NaN
1,0.932588,20005289,HUMIRA AC x 2 (40mg),"2 LAPICERAS PRELL. X 0,4 ml",8054083014050,"$ 1,480,292.66","$ 1,223,382.36","$ 2,146,424.35",humira ac x 2 40 mg lapiceras prell 04 ml,119550.0,...,101.0,E,0.0,NaN,ITGEV,0,0.0,ABBVIE,NaN,NaN
2,0.951788,20021980,HUMIRA AC x 1 (80 mg),"1 LAPICERA PRELL. X 0,8 ml",8054083017648,"$ 1,553,789.84","$ 1,284,123.82","$ 2,252,995.41",humira ac x 1 80 mg lapicera prell 08 ml,119550.0,...,101.0,E,0.0,NaN,ITGEV,0,0.0,ABBVIE,NaN,NaN
3,0.601083,20062091,RINVOQ 15MG,15 MG x 30 COMP REC DE LIB. PROLONGADA,8054083018706,"$ 1,799,606.71","$ 1,487,278.28","$ 2,609,429.58",rinvoq 15 mg x 30 comp rec de lib prolongada,128184.0,...,NaN,E,0.0,NaN,ITGEV,0.0,0.0,ABBVIE,NaN,NaN
4,0.593531,20069005,RINVOQ 30MG,30 MG x 30 COMP REC DE LIB. PROLONGADA,8054083021386,"$ 3,184,521.02","$ 2,631,835.62","$ 4,617,555.55",rinvoq 30 mg x comp rec de lib prolongada,124959.0,...,NaN,E,0.0,NaN,ITGEV,0.0,0.0,ABBVIE,NaN,NaN
5,0.623665,20078725,RINVOQ 45MG,45 MG x 28 COMP REC DE LIB. PROLONGADA,8054083023922,"$ 4,494,517.30","$ 3,714,477.10","$ 6,517,049.97",rinvoq 45 mg x 28 comp rec de lib prolongada,128184.0,...,NaN,E,0.0,NaN,ITGEV,0.0,0.0,ABBVIE,NaN,NaN
6,0.776140,20069017,SKYRIZI,1 LAPICERA PRELLENADA por 1 ML (150 mg),8054083021935,"$ 4,344,159.74","$ 3,590,214.67","$ 6,299,031.64",skyrizi 1 lapicera prellenada por ml 150 mg,124898.0,...,NaN,E,0.0,NaN,ITGEV,0.0,0.0,ABBVIE,NaN,NaN
7,0.886561,20079636,SKYRIZI,360mg /2.4ml Cart. prellenado x 1 600mg /10ml ...,8054083023311,"$ 4,561,367.72","$ 3,769,725.40","$ 6,613,983.22",skyrizi 360 mg 24 ml cart prellenado x 1 600 1...,130998.0,...,NaN,E,0.0,NaN,ITGEV,0.0,0.0,ABBVIE,NaN,NaN
8,0.582979,20079635,SKYRIZI,None,8054083023328,"$ 4,561,367.72","$ 3,769,725.40","$ 6,613,983.22",skyrizi,124898.0,...,NaN,E,0.0,NaN,ITGEV,0.0,0.0,ABBVIE,NaN,NaN
9,0.692462,20058578,KALETRA SOLUCIÓN,160 ML,8054083006406,"$ 148,177.93","$ 122,461.09","$ 214,857.98",kaletra solucin 160 ml,119555.0,...,101.0,E,0.0,NaN,ITGEV,0,0.0,ABBVIE,NaN,NaN


In [49]:
# Instanciamos el modelo de SentenceTransformer para obtener embeddings de texto
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Función para obtener embeddings de las descripciones
def get_embeddings(descriptions):
    return model.encode(descriptions.tolist())

# Comparar descripciones de productos utilizando embeddings
"""
    DESCRIPCIÓN: proviene del CSV PROVEEDOR 
    descrip1: proviene del CSV FILTRADO DE LA MASTER
"""

if df_Laboratorio_MasterProducto_Filtrado is not None:
    proveedor_embeddings = get_embeddings(df_resultado_PROVEEDOR['descrip1_descrip2_PROVEEDOR'])
    laboratorio_embeddings = get_embeddings(df_Laboratorio_MasterProducto_Filtrado['descrip1_descrip2_MasterProductos']) 


    matches = []

    for i, prov_row in df_resultado_PROVEEDOR.iterrows():
        prov_desc = prov_row['descrip1_descrip2_PROVEEDOR']

        # Calculamos la similitud de coseno entre el producto proveedor y los productos del laboratorio
        cosine_similarities = cosine_similarity([proveedor_embeddings[i]], laboratorio_embeddings)

        best_match_index = cosine_similarities.argmax()  # El índice de la mejor coincidencia
        best_match_score = cosine_similarities.max()  # El puntaje de similitud

        match_info = prov_row.to_dict()
        match_info.update(df_Laboratorio_MasterProducto_Filtrado.iloc[best_match_index].to_dict())
        match_info['similaridad'] = best_match_score
        matches.append(match_info)

    df_matches_2 = pd.DataFrame(matches)

    # Reordenamos las columnas para que la columna 'similaridad' esté al inicio
    cols = ['similaridad'] + [col for col in df_matches_2.columns if col != 'similaridad']
    df_matches_2 = df_matches_2[cols]

    print(f"\nCoincidencias encontradas: {len(df_matches_2[df_matches_2['similaridad'] > 0])}")
    print(df_matches_2)

    productos_sin_coincidencia = df_Laboratorio_MasterProducto_Filtrado[~df_Laboratorio_MasterProducto_Filtrado['descrip1_descrip2_MasterProductos'].isin(
        df_matches_2[df_matches_2['similaridad'] > 0]['descrip1_descrip2_MasterProductos']
    )]

    print(f"\nProductos del laboratorio sin coincidencias: {len(productos_sin_coincidencia)}")
    print(productos_sin_coincidencia)
else:
    print("No se encontraron coincidencias para el laboratorio especificado.")

df_matches_2



Coincidencias encontradas: 27
    similaridad Unnamed: 0                  DESCRIPCIÓN  \
0      0.817033   20015888         HUMIRA AC x 2 (20mg)   
1      0.867842   20005289        HUMIRA AC x 2 (40mg)    
2      0.903211   20021980        HUMIRA AC x 1 (80 mg)   
3      0.644030   20062091                 RINVOQ 15MG    
4      0.612940   20069005                  RINVOQ 30MG   
5      0.659430   20078725                  RINVOQ 45MG   
6      0.766860   20069017                      SKYRIZI   
7      0.802413   20079636                      SKYRIZI   
8      0.582979   20079635                      SKYRIZI   
9      0.744879   20058578             KALETRA SOLUCIÓN   
10     0.654478   20025520         KALETRA COMPRIMIDOS    
11     0.688451   20012232          KALETRA COMPRIMIDOS   
12     0.810732   20011289                    RITONAVIR   
13     0.716093   20011756                      MAVIRET   
14     0.652490   20019068   2 JERINGAS PRELL. X 0,2 ml   
15     0.578289   2001906

,similaridad,Unnamed: 0,DESCRIPCIÓN,PRESENTACION,CODIGO DE BARRAS,PRECIO CON IVA,PRECIO SIN IVA,PUBLICO\nSUGERIDO,descrip1_descrip2_PROVEEDOR,niprod,...,tipo_egreso,pc_diftrans,cod_cvta,cat_item,cant_rep,pto_rep,atrib0,atrib1,atrib2,descrip1_descrip2_MasterProductos
0,0.817033,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",8054083017068,"$ 740,145.59","$ 611,690.58","$ 1,073,211.02","HUMIRA AC x 2 (20mg) 2 JERINGAS PRELL. X 0,2 ml",119551.0,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE,NaN,NaN,humira ac x 2 20 mg 02 ml jerprell
1,0.867842,20005289,HUMIRA AC x 2 (40mg),"2 LAPICERAS PRELL. X 0,4 ml",8054083014050,"$ 1,480,292.66","$ 1,223,382.36","$ 2,146,424.35","HUMIRA AC x 2 (40mg) 2 LAPICERAS PRELL. X 0,4 ml",119550.0,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE,NaN,NaN,humira ac x 1 80 mg 08 ml lapprell
2,0.903211,20021980,HUMIRA AC x 1 (80 mg),"1 LAPICERA PRELL. X 0,8 ml",8054083017648,"$ 1,553,789.84","$ 1,284,123.82","$ 2,252,995.41","HUMIRA AC x 1 (80 mg) 1 LAPICERA PRELL. X 0,8 ml",119550.0,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE,NaN,NaN,humira ac x 1 80 mg 08 ml lapprell
3,0.644030,20062091,RINVOQ 15MG,15 MG x 30 COMP REC DE LIB. PROLONGADA,8054083018706,"$ 1,799,606.71","$ 1,487,278.28","$ 2,609,429.58",RINVOQ 15MG 15 MG x 30 COMP REC DE LIB. PROLO...,128184.0,...,E,0.0,NaN,ITGEV,0.0,0.0,ABBVIE,NaN,NaN,rinvoq 45 mg complibprol x 28
4,0.612940,20069005,RINVOQ 30MG,30 MG x 30 COMP REC DE LIB. PROLONGADA,8054083021386,"$ 3,184,521.02","$ 2,631,835.62","$ 4,617,555.55",RINVOQ 30MG 30 MG x 30 COMP REC DE LIB. PROLON...,128184.0,...,E,0.0,NaN,ITGEV,0.0,0.0,ABBVIE,NaN,NaN,rinvoq 45 mg complibprol x 28
5,0.659430,20078725,RINVOQ 45MG,45 MG x 28 COMP REC DE LIB. PROLONGADA,8054083023922,"$ 4,494,517.30","$ 3,714,477.10","$ 6,517,049.97",RINVOQ 45MG 45 MG x 28 COMP REC DE LIB. PROLON...,128184.0,...,E,0.0,NaN,ITGEV,0.0,0.0,ABBVIE,NaN,NaN,rinvoq 45 mg complibprol x 28
6,0.766860,20069017,SKYRIZI,1 LAPICERA PRELLENADA por 1 ML (150 mg),8054083021935,"$ 4,344,159.74","$ 3,590,214.67","$ 6,299,031.64",SKYRIZI 1 LAPICERA PRELLENADA por 1 ML (150 mg),124898.0,...,E,0.0,NaN,ITGEV,0.0,0.0,ABBVIE,NaN,NaN,skyrizi 150 mg lapprell x 1
7,0.802413,20079636,SKYRIZI,360mg /2.4ml Cart. prellenado x 1 600mg /10ml ...,8054083023311,"$ 4,561,367.72","$ 3,769,725.40","$ 6,613,983.22",SKYRIZI 360mg /2.4ml Cart. prellenado x 1 600m...,130998.0,...,E,0.0,NaN,ITGEV,0.0,0.0,ABBVIE,NaN,NaN,skyrizi 600 mg vial x 1
8,0.582979,20079635,SKYRIZI,None,8054083023328,"$ 4,561,367.72","$ 3,769,725.40","$ 6,613,983.22",SKYRIZI,124898.0,...,E,0.0,NaN,ITGEV,0.0,0.0,ABBVIE,NaN,NaN,skyrizi 150 mg lapprell x 1
9,0.744879,20058578,KALETRA SOLUCIÓN,160 ML,8054083006406,"$ 148,177.93","$ 122,461.09","$ 214,857.98",KALETRA SOLUCIÓN 160 ML,119555.0,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE,NaN,NaN,kaletra 25100 mg comprec x 60


In [50]:
df_resultado_PROVEEDOR

,Unnamed: 0,DESCRIPCIÓN,PRESENTACION,CODIGO DE BARRAS,PRECIO CON IVA,PRECIO SIN IVA,PUBLICO\nSUGERIDO,descrip1_descrip2_PROVEEDOR
0,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",8054083017068,"$ 740,145.59","$ 611,690.58","$ 1,073,211.02","HUMIRA AC x 2 (20mg) 2 JERINGAS PRELL. X 0,2 ml"
1,20005289,HUMIRA AC x 2 (40mg),"2 LAPICERAS PRELL. X 0,4 ml",8054083014050,"$ 1,480,292.66","$ 1,223,382.36","$ 2,146,424.35","HUMIRA AC x 2 (40mg) 2 LAPICERAS PRELL. X 0,4 ml"
2,20021980,HUMIRA AC x 1 (80 mg),"1 LAPICERA PRELL. X 0,8 ml",8054083017648,"$ 1,553,789.84","$ 1,284,123.82","$ 2,252,995.41","HUMIRA AC x 1 (80 mg) 1 LAPICERA PRELL. X 0,8 ml"
3,20062091,RINVOQ 15MG,15 MG x 30 COMP REC DE LIB. PROLONGADA,8054083018706,"$ 1,799,606.71","$ 1,487,278.28","$ 2,609,429.58",RINVOQ 15MG 15 MG x 30 COMP REC DE LIB. PROLO...
4,20069005,RINVOQ 30MG,30 MG x 30 COMP REC DE LIB. PROLONGADA,8054083021386,"$ 3,184,521.02","$ 2,631,835.62","$ 4,617,555.55",RINVOQ 30MG 30 MG x 30 COMP REC DE LIB. PROLON...
5,20078725,RINVOQ 45MG,45 MG x 28 COMP REC DE LIB. PROLONGADA,8054083023922,"$ 4,494,517.30","$ 3,714,477.10","$ 6,517,049.97",RINVOQ 45MG 45 MG x 28 COMP REC DE LIB. PROLON...
6,20069017,SKYRIZI,1 LAPICERA PRELLENADA por 1 ML (150 mg),8054083021935,"$ 4,344,159.74","$ 3,590,214.67","$ 6,299,031.64",SKYRIZI 1 LAPICERA PRELLENADA por 1 ML (150 mg)
7,20079636,SKYRIZI,360mg /2.4ml Cart. prellenado x 1 600mg /10ml ...,8054083023311,"$ 4,561,367.72","$ 3,769,725.40","$ 6,613,983.22",SKYRIZI 360mg /2.4ml Cart. prellenado x 1 600m...
8,20079635,SKYRIZI,None,8054083023328,"$ 4,561,367.72","$ 3,769,725.40","$ 6,613,983.22",SKYRIZI
9,20058578,KALETRA SOLUCIÓN,160 ML,8054083006406,"$ 148,177.93","$ 122,461.09","$ 214,857.98",KALETRA SOLUCIÓN 160 ML


In [51]:
df_Laboratorio_MasterProducto_Filtrado

,niprod,cod_prod,cod_abrev,descrip1,descrip2,abrev,cod_catprod,codbarra,cod_fbarra,unid_v,...,tipo_egreso,pc_diftrans,cod_cvta,cat_item,cant_rep,pto_rep,atrib0,atrib1,atrib2,descrip1_descrip2_MasterProductos
787,23422.0,38945,38945,ACULAR LS sol.oft.x 5 ml,NaN,ACULAR LS sol.oft.x,PT,8054083026008,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE ALLERGAN,NaN,NaN,acular ls soloft x 5 ml
788,23421.0,15467,15467,ACULAR sol.oft.x 5 ml,NaN,ACULAR sol.oft.x 5 m,PT,8054083026329,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE ALLERGAN,NaN,NaN,acular soloft x 5 ml
790,23423.0,48009,48009,ACUVAIL viales unids.x 30 x0.4ml,NaN,ACUVAIL viales unids,PT,7795316000099,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE ALLERGAN,NaN,NaN,acuvail viales unids x 30 x04 ml
1654,23424.0,44172,44172,AIBREX sol.oft.x 5 ml,NaN,AIBREX sol.oft.x 5 m,PT,7795316000112,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE ALLERGAN,NaN,NaN,aibre x soloft 5 ml
2439,23427.0,31189,31189,ALPHAGAN P sol.oft.x 5 ml,NaN,ALPHAGAN P sol.oft.x,PT,8054083026312,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE ALLERGAN,NaN,NaN,alphagan p soloft x 5 ml
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56458,119578.0,39616,39616,ZEMPLAR 5 mcg a.x 5 x 1 ml,NaN,ZEMPLAR 5 mcg a.x 5,PT,8054083003566.0,ABBOTT6,UN,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE,NaN,NaN,zemplar 5 mcg a x x1 ml
56459,119579.0,54285,54285,ZEMPLAR 5 mcg vial x 5 x 1 ml,NaN,ZEMPLAR 5 mcg vial x,PT,8054083017907.0,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE,NaN,NaN,zemplar 5 mcg vial x x1 ml
56745,23480.0,34168,34168,ZYMARAN 0.3% sol.oft.x 5 ml,NaN,ZYMARAN 0.3% sol.oft,PT,8054083026336.0,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE ALLERGAN,NaN,NaN,zymaran 03% soloft x 5 ml
56746,23481.0,47867,47867,ZYMAXID fco.gotero x 5 ml,NaN,ZYMAXID fco.gotero x,PT,8054083026350.0,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE ALLERGAN,NaN,NaN,zymaxid fcogotero x 5 ml


In [52]:
# Suponiendo que tienes los DataFrames df_matches_2 y df_matches
df1 = df_matches_2[["similaridad", "DESCRIPCIÓN", "descrip1"]]
df2 = df_matches[["similaridad"]]  # Asegúrate de que "similaridad" también esté entre corchetes


# Concatenar ambos DataFrames a lo largo del eje de las columnas (axis=1)
df_combined_similitud = pd.concat([df1, df2], axis=1)

# Imprimir el DataFrame combinado
df_combined_similitud


,similaridad,DESCRIPCIÓN,descrip1,similaridad
0,0.817033,HUMIRA AC x 2 (20mg),HUMIRA AC X 2 (20 MG) 20mg/0.2ml jer.prell.x 2,0.911634
1,0.867842,HUMIRA AC x 2 (40mg),HUMIRA AC X 1 (80 MG) 80mg/0.8ml lap.prell.x 1,0.932588
2,0.903211,HUMIRA AC x 1 (80 mg),HUMIRA AC X 1 (80 MG) 80mg/0.8ml lap.prell.x 1,0.951788
3,0.644030,RINVOQ 15MG,RINVOQ 45 mg comp.lib.prol.x 28,0.601083
4,0.612940,RINVOQ 30MG,RINVOQ 45 mg comp.lib.prol.x 28,0.593531
5,0.659430,RINVOQ 45MG,RINVOQ 45 mg comp.lib.prol.x 28,0.623665
6,0.766860,SKYRIZI,SKYRIZI 150 mg lap.prell. x 1,0.776140
7,0.802413,SKYRIZI,SKYRIZI 600 mg vial x 1,0.886561
8,0.582979,SKYRIZI,SKYRIZI 150 mg lap.prell. x 1,0.582979
9,0.744879,KALETRA SOLUCIÓN,KALETRA 25/100 mg comp.rec.x 60,0.692462


In [53]:


df_Laboratorio_MasterProducto_Filtrado_match3  = df_Laboratorio_MasterProducto_Filtrado.copy()
# Crear la nueva columna 'descrip1_descrip2' combinando las columnas 'descrip1' y 'descrip2'
df_Laboratorio_MasterProducto_Filtrado_match3["descrip1_descrip2_MasterProductos"] = df_Laboratorio_MasterProducto_Filtrado_match3["descrip1"].fillna('') + " " + df_Laboratorio_MasterProducto_Filtrado_match3["descrip2"].fillna('')

# Mostrar el resultado

# Cargar el CSV del proveedor procesado 
ruta_csv = "prueba2.csv"
df_resultado_PROVEEDOR_match3 = procesar_csv(ruta_csv).copy()
df_resultado_PROVEEDOR_match3



# Asegurarse de manejar valores nulos antes de concatenar
df_resultado_PROVEEDOR_match3["descrip1_descrip2_PROVEEDOR"] = df_resultado_PROVEEDOR_match3["DESCRIPCIÓN"].fillna('') + " " + df_resultado_PROVEEDOR_match3["PRESENTACION"].fillna('')
df_resultado_PROVEEDOR_match3["descrip1_descrip2_PROVEEDOR"] = df_resultado_PROVEEDOR_match3["descrip1_descrip2_PROVEEDOR"].str.strip()  # Eliminar espacios en los extremos

df_Laboratorio_MasterProducto_Filtrado_match3


Usando el módulo: VARIOS PRODUCTOS EN UNA FILA


,niprod,cod_prod,cod_abrev,descrip1,descrip2,abrev,cod_catprod,codbarra,cod_fbarra,unid_v,...,tipo_egreso,pc_diftrans,cod_cvta,cat_item,cant_rep,pto_rep,atrib0,atrib1,atrib2,descrip1_descrip2_MasterProductos
787,23422.0,38945,38945,ACULAR LS sol.oft.x 5 ml,NaN,ACULAR LS sol.oft.x,PT,8054083026008,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE ALLERGAN,NaN,NaN,ACULAR LS sol.oft.x 5 ml
788,23421.0,15467,15467,ACULAR sol.oft.x 5 ml,NaN,ACULAR sol.oft.x 5 m,PT,8054083026329,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE ALLERGAN,NaN,NaN,ACULAR sol.oft.x 5 ml
790,23423.0,48009,48009,ACUVAIL viales unids.x 30 x0.4ml,NaN,ACUVAIL viales unids,PT,7795316000099,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE ALLERGAN,NaN,NaN,ACUVAIL viales unids.x 30 x0.4ml
1654,23424.0,44172,44172,AIBREX sol.oft.x 5 ml,NaN,AIBREX sol.oft.x 5 m,PT,7795316000112,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE ALLERGAN,NaN,NaN,AIBREX sol.oft.x 5 ml
2439,23427.0,31189,31189,ALPHAGAN P sol.oft.x 5 ml,NaN,ALPHAGAN P sol.oft.x,PT,8054083026312,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE ALLERGAN,NaN,NaN,ALPHAGAN P sol.oft.x 5 ml
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56458,119578.0,39616,39616,ZEMPLAR 5 mcg a.x 5 x 1 ml,NaN,ZEMPLAR 5 mcg a.x 5,PT,8054083003566.0,ABBOTT6,UN,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE,NaN,NaN,ZEMPLAR 5 mcg a.x 5 x 1 ml
56459,119579.0,54285,54285,ZEMPLAR 5 mcg vial x 5 x 1 ml,NaN,ZEMPLAR 5 mcg vial x,PT,8054083017907.0,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE,NaN,NaN,ZEMPLAR 5 mcg vial x 5 x 1 ml
56745,23480.0,34168,34168,ZYMARAN 0.3% sol.oft.x 5 ml,NaN,ZYMARAN 0.3% sol.oft,PT,8054083026336.0,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE ALLERGAN,NaN,NaN,ZYMARAN 0.3% sol.oft.x 5 ml
56746,23481.0,47867,47867,ZYMAXID fco.gotero x 5 ml,NaN,ZYMAXID fco.gotero x,PT,8054083026350.0,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE ALLERGAN,NaN,NaN,ZYMAXID fco.gotero x 5 ml


In [54]:
# Instanciamos el modelo de SentenceTransformer para obtener embeddings de texto
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Función para obtener embeddings de las descripciones
def get_embeddings(descriptions):
    return model.encode(descriptions.tolist())

# Comparar descripciones de productos utilizando embeddings
"""
    DESCRIPCIÓN: proviene del CSV PROVEEDOR 
    descrip1: proviene del CSV FILTRADO DE LA MASTER
"""

if df_Laboratorio_MasterProducto_Filtrado_match3 is not None:
    proveedor_embeddings = get_embeddings(df_resultado_PROVEEDOR_match3['descrip1_descrip2_PROVEEDOR'])
    laboratorio_embeddings = get_embeddings(df_Laboratorio_MasterProducto_Filtrado_match3['descrip1_descrip2_MasterProductos'])

    matches = []

    for i, prov_row in df_resultado_PROVEEDOR_match3.iterrows():
        prov_desc = prov_row['descrip1_descrip2_PROVEEDOR']

        # Calculamos la similitud de coseno entre el producto proveedor y los productos del laboratorio
        cosine_similarities = cosine_similarity([proveedor_embeddings[i]], laboratorio_embeddings)

        best_match_index = cosine_similarities.argmax()  # El índice de la mejor coincidencia
        best_match_score = cosine_similarities.max()  # El puntaje de similitud

        match_info = prov_row.to_dict()
        match_info.update(df_Laboratorio_MasterProducto_Filtrado_match3.iloc[best_match_index].to_dict())
        match_info['similaridad'] = best_match_score
        matches.append(match_info)

    df_matches_3 = pd.DataFrame(matches)

    # Reordenamos las columnas para que la columna 'similaridad' esté al inicio
    cols = ['similaridad'] + [col for col in df_matches_3.columns if col != 'similaridad']
    df_matches_3 = df_matches_3[cols]

    print(f"\nCoincidencias encontradas: {len(df_matches_3[df_matches_3['similaridad'] > 0])}")
    print(df_matches_3)

    productos_sin_coincidencia = df_Laboratorio_MasterProducto_Filtrado_match3[~df_Laboratorio_MasterProducto_Filtrado_match3['descrip1_descrip2_MasterProductos'].isin(
        df_matches_3[df_matches_3['similaridad'] > 0]['descrip1_descrip2_MasterProductos']
    )]

    print(f"\nProductos del laboratorio sin coincidencias: {len(productos_sin_coincidencia)}")
    print(productos_sin_coincidencia)
else:
    print("No se encontraron coincidencias para el laboratorio especificado.")




Coincidencias encontradas: 27
    similaridad Unnamed: 0                  DESCRIPCIÓN  \
0      0.904081   20015888         HUMIRA AC x 2 (20mg)   
1      0.883245   20005289        HUMIRA AC x 2 (40mg)    
2      0.912229   20021980        HUMIRA AC x 1 (80 mg)   
3      0.833805   20062091                 RINVOQ 15MG    
4      0.823783   20069005                  RINVOQ 30MG   
5      0.844630   20078725                  RINVOQ 45MG   
6      0.751093   20069017                      SKYRIZI   
7      0.802413   20079636                      SKYRIZI   
8      0.613015   20079635                      SKYRIZI   
9      0.729983   20058578             KALETRA SOLUCIÓN   
10     0.684076   20025520         KALETRA COMPRIMIDOS    
11     0.720336   20012232          KALETRA COMPRIMIDOS   
12     0.716444   20011289                    RITONAVIR   
13     0.746770   20011756                      MAVIRET   
14     0.658348   20019068   2 JERINGAS PRELL. X 0,2 ml   
15     0.588098   2001906

In [55]:


# Suponiendo que tienes los DataFrames df_matches, df_matches_2 y df_matches_3
df1 = df_matches[["similaridad", "niprod"]]
df2 = df_matches_2[["similaridad", "niprod"]]
df3 = df_matches_3[["similaridad", "descrip1_descrip2_MasterProductos", "descrip1_descrip2_PROVEEDOR", "niprod"]]

# Concatenar los DataFrames a lo largo del eje de las columnas (axis=1)
df_combined_similitud = pd.concat([df1, df2, df3], axis=1)

# Renombrar las columnas de similaridad de cada DataFrame para diferenciarlas
df_combined_similitud.columns = ['similaridad_1', "niprod", 'similaridad_2',"niprod", "similaridad_3",  "descrip1_descrip2_MasterProductos", "descrip1_descrip2_PROVEEDOR", "niprod"]

# Imprimir el DataFrame combinado con los nuevos nombres de columna
df_combined_similitud.head(20)


,similaridad_1,niprod,similaridad_2,niprod,similaridad_3,descrip1_descrip2_MasterProductos,descrip1_descrip2_PROVEEDOR,niprod
0,0.911634,119551.0,0.817033,119551.0,0.904081,HUMIRA AC X 2 (20 MG) 20mg/0.2ml jer.prell.x 2,"HUMIRA AC x 2 (20mg) 2 JERINGAS PRELL. X 0,2 ml",119551.0
1,0.932588,119550.0,0.867842,119550.0,0.883245,HUMIRA AC X 1 (80 MG) 80mg/0.8ml lap.prell.x 1,"HUMIRA AC x 2 (40mg) 2 LAPICERAS PRELL. X 0,4 ml",119550.0
2,0.951788,119550.0,0.903211,119550.0,0.912229,HUMIRA AC X 1 (80 MG) 80mg/0.8ml lap.prell.x 1,"HUMIRA AC x 1 (80 mg) 1 LAPICERA PRELL. X 0,8 ml",119550.0
3,0.601083,128184.0,0.644030,128184.0,0.833805,RINVOQ 45 mg comp.lib.prol.x 28,RINVOQ 15MG 15 MG x 30 COMP REC DE LIB. PROLO...,128184.0
4,0.593531,124959.0,0.612940,128184.0,0.823783,RINVOQ 45 mg comp.lib.prol.x 28,RINVOQ 30MG 30 MG x 30 COMP REC DE LIB. PROLON...,128184.0
5,0.623665,128184.0,0.659430,128184.0,0.844630,RINVOQ 45 mg comp.lib.prol.x 28,RINVOQ 45MG 45 MG x 28 COMP REC DE LIB. PROLON...,128184.0
6,0.776140,124898.0,0.766860,124898.0,0.751093,SKYRIZI 150 mg lap.prell. x 1,SKYRIZI 1 LAPICERA PRELLENADA por 1 ML (150 mg),124898.0
7,0.886561,130998.0,0.802413,130998.0,0.802413,SKYRIZI 600 mg vial x 1,SKYRIZI 360mg /2.4ml Cart. prellenado x 1 600m...,130998.0
8,0.582979,124898.0,0.582979,124898.0,0.613015,SKYRIZI 150 mg lap.prell. x 1,SKYRIZI,124898.0
9,0.692462,119555.0,0.744879,119555.0,0.729983,KALETRA 25/100 mg comp.rec.x 60,KALETRA SOLUCIÓN 160 ML,119555.0
